In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import itertools

In [2]:
names = ['mobile','touchpoint_id','action_time','row_number','time_lag','tp_id_through','row_number_lag','conv_period']
df = pd.read_csv('data/tp_analysis_base.csv',sep = '\t', names = names)
df['mobile'] = df['mobile'].astype(str)

In [3]:
id_mapping = pd.read_csv('data/id_mapping.csv')

In [4]:
tp_name = {k:v for k,v in zip(id_mapping.touchpoint_id,id_mapping.touchpoint_name)}
inbound_map = {k:v for k,v in zip(id_mapping.touchpoint_id,id_mapping.is_inbound)}

In [2]:
# fetch other information
# deal_time.csv 为从marketing_modeling.dw_deliver_behavior获取的mobile and behavior_time
deal_date = pd.read_csv('data/deal_time.csv',names=['mobile','deal_time'],sep='\t')

#### Segmentation

In [7]:
cust_user = df[(df.touchpoint_id == '004000000000_tp') | (df.touchpoint_id == '005000000000_tp')].mobile.unique().tolist()

In [8]:
# segment
df['segment'] = np.nan
df['segment'][df.mobile.isin(deal_date.mobile.unique().tolist())] = 'deal'
df['segment'][(df.mobile.isin(cust_user)) & (df.segment.isna()==True)] = 'cust'
df['segment'] = df['segment'].fillna('potential')

/app/anaconda3/envs/python37_smcsandbox/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
df[['mobile','segment']].drop_duplicates().segment.value_counts()

#### Potential

In [ ]:
potential_df = df[df.segment == 'potential']
potential_df = potential_df[['mobile','touchpoint_id','action_time','row_number']]

potential_df['row_number_to'] = potential_df['row_number'].apply(lambda x:x+1)
potential_df = potential_df.merge(potential_df[['mobile','touchpoint_id','action_time','row_number']]\
                                  .rename(columns={'touchpoint_id':'to_id','action_time':'to_time'}),
                                  left_on=['mobile','row_number_to'],right_on=['mobile','row_number'],how='left')

potential_df['fr_tp'] = potential_df['touchpoint_id'].apply(lambda x:tp_name[x])
potential_df['to_tp'] = [tp_name[x] if type(x) != float else 'Exit' for x in potential_df['to_id']]

potential_result = potential_df.groupby(by=['fr_tp','to_tp'],as_index=False).agg({'mobile':'count'})
#potential_result.to_csv('potential_exit_result.csv',index=False)

In [ ]:
potential_result_g = potential_result.groupby(by='fr_tp',as_index=False).agg({'mobile':'sum'}).rename(columns={'mobile':'Total_traffic'})
potential_result = potential_result.merge(potential_result_g,on='fr_tp',how='left')
potential_result['transition_rate'] = potential_result['mobile']/potential_result['Total_traffic']
potential_result.to_csv('potential_exit_result.csv')

#### Customers

In [52]:
cust_df = df[df.segment == 'cust']
cust_df = cust_df[['mobile','touchpoint_id','action_time','row_number']]

In [53]:
oppor_time = cust_df[(cust_df.touchpoint_id == '004000000000_tp')][['mobile','touchpoint_id','action_time','row_number']]\
.rename(columns={'action_time':'oppor_time','row_number':'oppor_loc','touchpoint_id':'oppor_tp'})

followup_time = cust_df[cust_df.touchpoint_id.str.contains('009002')][['mobile','touchpoint_id','action_time','row_number']]\
.rename(columns={'action_time':'followup_time','row_number':'followup_loc','touchpoint_id':'followup_tp'})

cust_time = cust_df[cust_df.touchpoint_id == '005000000000_tp'][['mobile','touchpoint_id','action_time','row_number']]\
.rename(columns={'action_time':'cust_time','row_number':'cust_loc','touchpoint_id':'cust_tp'})

In [57]:
tmp = oppor_time.merge(followup_time,left_on=['mobile','oppor_time'],right_on=['mobile','followup_time'])\
.merge(cust_time,left_on=['mobile','oppor_time'],right_on=['mobile','cust_time'])

tmp['max'] = tmp[['oppor_loc','followup_loc','cust_loc']].max(axis=1)
tmp['min'] = tmp[['oppor_loc','followup_loc','cust_loc']].min(axis=1)

tmp['oppor_loc'] = tmp['min']
tmp['cust_loc'] = tmp['min']
tmp['followup_loc'] = tmp['max']

df1 = cust_df[(cust_df.touchpoint_id != '004000000000_tp') & (cust_df.touchpoint_id.str.contains('009002') == False)\
        & (cust_df.touchpoint_id != '005000000000_tp')]
df2 = tmp[['mobile','oppor_tp','oppor_time','oppor_loc']]
df3 = tmp[['mobile','followup_tp','followup_time','followup_loc']]
df4 = tmp[['mobile','cust_tp','cust_time','cust_loc']]

df2.columns = ['mobile','touchpoint_id','action_time','row_number']
df3.columns = ['mobile','touchpoint_id','action_time','row_number']
df4.columns = ['mobile','touchpoint_id','action_time','row_number']

cust_df = df1.append(df2).append(df3).append(df4)

In [61]:
cust_df['row_number_to'] = cust_df['row_number'].apply(lambda x:x+1)
cust_df = cust_df.merge(cust_df[['mobile','touchpoint_id','action_time','row_number']]\
                                  .rename(columns={'touchpoint_id':'to_id','action_time':'to_time'}),
                                  left_on=['mobile','row_number_to'],right_on=['mobile','row_number'],how='left')
cust_df['fr_tp'] = cust_df['touchpoint_id'].apply(lambda x:tp_name[x])
cust_df['to_tp'] = [tp_name[x] if type(x) != float else 'Exit' for x in cust_df['to_id']]

In [64]:
max_index = cust_df.groupby(by='mobile',as_index=False).agg({'row_number_x':'max'})
max_index.columns = ['mobile','max_index']

cust_df = cust_df.merge(max_index,on='mobile',how='left')

drop = cust_df[(cust_df.to_tp == 'Exit') & (cust_df.max_index != cust_df.row_number_x)].index.tolist()
cust_df = cust_df[cust_df.index.isin(drop) == False]

In [79]:
cust_result = cust_df.groupby(by=['fr_tp','to_tp'],as_index=False).agg({'mobile':'count'})
cust_result.to_csv('cust_exit_result.csv',index=False)

cust_result_g = cust_result.groupby(by = 'fr_tp',as_index=False).agg({'mobile':'sum'}).rename(columns={'mobile':'Total_traffic'})
cust_result = cust_result.merge(cust_result_g,on='fr_tp',how='left')
cust_result['transition_rate'] = cust_result['mobile']/cust_result['Total_traffic']
cust_result.to_csv('cust_exit_result.csv')

#### Deal

In [ ]:
deal_df = df[df.segment == 'deal']
deal_df = deal_df[['mobile','touchpoint_id','action_time','row_number']]

deal_df['row_number_to'] = deal_df['row_number'].apply(lambda x:x+1)
deal_df = deal_df.merge(deal_df[['mobile','touchpoint_id','action_time','row_number']]\
                                  .rename(columns={'touchpoint_id':'to_id','action_time':'to_time'}),
                                  left_on=['mobile','row_number_to'],right_on=['mobile','row_number'],how='left')
deal_df['fr_tp'] = deal_df['touchpoint_id'].apply(lambda x:tp_name[x])
deal_df['to_tp'] = [tp_name[x] if type(x) != float else 'Exit' for x in deal_df['to_id']]

In [ ]:
deal_result = deal_df.groupby(by=['fr_tp','to_tp'],as_index=False).agg({'mobile':'count'})
#deal_result.to_csv('deal_exit_result.csv',index=False)

deal_result_g = deal_result.groupby(by = 'fr_tp',as_index=False).agg({'mobile':'sum'}).rename(columns={'mobile':'Total_traffic'})
deal_result = deal_result.merge(deal_result_g,on='fr_tp',how='left')
deal_result['transition_rate'] = deal_result['mobile']/deal_result['Total_traffic']
deal_result.to_csv('deal_exit_result.csv')